run with tensorlab env

In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import matplotlib.pyplot as plt
from tflearn.data_utils import shuffle
from numpy import genfromtxt
%matplotlib inline

In [2]:
merchants_tpvs = genfromtxt('merchants_tpvs3.csv', delimiter=',')

In [3]:
shuf = merchants_tpvs
np.random.shuffle(shuf)

In [4]:
data_in_60_x = np.array(shuf[0][0:60])
data_in_60_y = np.array(shuf[0][60])
for i in shuf:
    for j in range(len(i) - 60):
        data_in_60_x = np.append( data_in_60_x, i[j: j+60])
        data_in_60_y = np.append(data_in_60_y, i[j+60])
        
# len(data_in_60)

In [5]:
data_in_60_x = np.reshape(data_in_60_x, (18501, 60))
len(data_in_60_x)

18501

In [6]:
test_fraction = 0.9
records = len(data_in_60_x)

train_X, test_X = data_in_60_x[:int(records*test_fraction)], data_in_60_x[int(records*test_fraction):]

In [7]:
train_Y, test_Y = data_in_60_y[:int(records*test_fraction)], data_in_60_y[int(records*test_fraction):]

In [8]:
# train_Y = np.array(train_X[0][244])
# for i in train_X[1:]:
#     train_Y = np.append( train_Y, i[244])
# #train_Y

In [9]:
# test_Y = np.array(test_X[0][244])
# for i in test_X[1:]:
#     test_Y = np.append( test_Y, i[244])

In [10]:
# new_train_X = np.array(train_X[0][:-1])
# for i in train_X[1:]:
#     new_train_X = np.append( new_train_X, i[:-1])


In [11]:
# new_test_X = np.array(test_X[0][:-1])
# for i in test_X[1:]:
#     new_test_X = np.append( new_test_X, i[:-1])

In [12]:
# new_test_X[0]

In [13]:
np.shape(test_Y)

(1851,)

In [14]:
# train_X = np.reshape(train_X, (16650, 60,1))

# train_Y = np.reshape(train_Y, (16650,1))

# test_X = np.reshape(test_X, (1851, 60,1))
# test_Y = np.reshape(test_Y, (1851,1))

In [20]:
train_X = np.reshape(train_X, (16650, 60,1))/1000

train_Y = np.reshape(train_Y, (16650,1))/1000

test_X = np.reshape(test_X, (1851, 60,1))/1000
test_Y = np.reshape(test_Y, (1851,1))/1000

In [21]:
# new_train_X = np.reshape(new_train_X, (90, 244,1))/1000
# train_Y = np.reshape(train_Y, (90,1))/1000
# new_test_X = np.reshape(new_test_X, (10, 244,1))/1000
# test_Y = np.reshape(test_Y, (10,1))/1000

In [28]:
# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()

    # input layer
    net = tflearn.input_data(shape=[None, 60,1])
    #net = tflearn.layers.normalization.batch_normalization (net, beta=0.0, gamma=1.0, epsilon=1e-05, decay=0.9, stddev=0.002, trainable=True, restore=True, reuse=False, scope=None, name='BatchNormalization')
    #net = tflearn.embedding(net, input_dim=1, output_dim=60)
    # net = tflearn.lstm(net, n_units=30, return_seq=True)
    net = tflearn.lstm(net, n_units=128, return_seq=False)
    net = tflearn.fully_connected(net, 254, activation='linear')
    net = tflearn.fully_connected(net, 64, activation='linear')
    net  = tflearn.layers.core.dropout (net, 0.5, noise_shape=None, name='Dropout')
    # net = tflearn.fully_connected(net, 32, activation='linear')
    net = tflearn.fully_connected(net, 1, activation='linear')
    
    #Output Layer
    adam = tflearn.optimizers.Adam(learning_rate=0.00001, beta1=0.99)
    net = tflearn.regression(net, optimizer= adam, loss='mean_square')
    
    # This model assumes that your network is named "net"  
    #model = tflearn.DNN(net, clip_gradients=0.0, tensorboard_verbose=0)
    model = tflearn.DNN(net, clip_gradients=5.0)
    #model = tflearn.DNN(net)
    return model

In [29]:
model = build_model()


In [30]:
# Training
model.fit(train_X, train_Y, n_epoch=10, validation_set=0.1, show_metric=True, batch_size=10, shuffle=True)
#model.fit(train_features, train_next_point, n_epoch=1, validation_set=0.1, show_metric=True, batch_size=1)

Training Step: 14989  | total loss: 2242.91846 | time: 113.978s
| Adam | epoch: 010 | loss: 2242.91846 - binary_acc: 0.0076 -- iter: 14980/14985
Training Step: 14990  | total loss: 2035.21472 | time: 116.373s
| Adam | epoch: 010 | loss: 2035.21472 - binary_acc: 0.0169 | val_loss: 1647.90003 - val_acc: 0.0180 -- iter: 14985/14985
--


In [25]:
# Compare the labels that our model predicts with the actual labels

# Find the indices of the most confident prediction for each item. That tells us the predicted digit for that sample.
predictions = np.array(model.predict(test_X))

# Calculate the accuracy, which is the percentage of times the predicated labels matched the actual labels
actual = test_Y
test_accuracy = np.mean(np.abs(predictions - actual), axis=0)

# Print out the result
print("Test accuracy: ", test_accuracy)

Test accuracy:  [ 25.95959738]


In [26]:
predictions[:50]

array([[ 62.55971909],
       [ 34.64829636],
       [ 31.80964279],
       [ 31.59939384],
       [ 36.70702362],
       [ 34.09413147],
       [ 31.46787071],
       [ 33.6192131 ],
       [ 34.05064392],
       [ 33.15245819],
       [ 32.44073868],
       [ 35.59375381],
       [ 36.73429489],
       [ 33.29591751],
       [ 32.68295288],
       [ 30.40307617],
       [ 29.95929146],
       [ 29.07912064],
       [ 26.88911629],
       [ 28.04605293],
       [ 25.35542107],
       [ 25.67716026],
       [ 25.07242012],
       [ 23.2420826 ],
       [ 24.14607811],
       [ 22.26185417],
       [ 25.94545555],
       [ 23.55274773],
       [ 40.47037125],
       [ 35.09722519],
       [ 37.30059433],
       [ 39.66332626],
       [ 37.22734451],
       [ 46.90947342],
       [ 41.09922409],
       [ 44.43722153],
       [ 42.32503128],
       [ 51.37192154],
       [ 46.40752029],
       [ 43.390625  ],
       [ 46.36268616],
       [ 41.47251892],
       [ 42.55068207],
       [ 41

In [27]:
actual[:50]

array([[ 362.417],
       [   1.3  ],
       [  27.35 ],
       [ 112.25 ],
       [  18.9  ],
       [   0.   ],
       [  60.701],
       [  49.376],
       [  29.05 ],
       [  27.   ],
       [  82.575],
       [  64.8  ],
       [   0.   ],
       [  23.74 ],
       [   4.45 ],
       [  25.1  ],
       [  21.45 ],
       [   1.75 ],
       [  43.82 ],
       [   0.   ],
       [  30.165],
       [  21.3  ],
       [   0.   ],
       [  35.8  ],
       [   0.   ],
       [  75.75 ],
       [   0.   ],
       [ 282.43 ],
       [   7.7  ],
       [  61.9  ],
       [  73.4  ],
       [  17.15 ],
       [ 171.   ],
       [   0.   ],
       [  85.5  ],
       [  31.34 ],
       [ 172.28 ],
       [  19.8  ],
       [  12.63 ],
       [  85.95 ],
       [   0.   ],
       [  59.13 ],
       [  38.18 ],
       [   0.   ],
       [ 343.4  ],
       [ 334.7  ],
       [  25.05 ],
       [   0.   ],
       [  11.55 ],
       [   1.37 ]])